In [1]:
%load_ext autoreload
%autoreload

import numpy as np
import pandas as pd
import os
import sys
import pickle

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from custom_scripts.config import loader
from custom_scripts.evaluate_performance import evaluate_performance
from custom_scripts.prepare_data import prepare_data

from sklearn.decomposition import PCA

from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor

### Load the data

In [15]:
day_of_week='Monday'

data_file= loader(day_of_week=day_of_week,data_type='processed',subtype='Normalized')
df=pd.read_pickle(data_file)

## Preparing the Dataset for Anomaly Detection 

Here we need to separate our class variable, which in our case is "Label" from the rest of the dataset.

Anomaly detection algorithms output +1 for inlier and -1 for outliers. Therefore:

1. We need to map multiple classes to binary classes.
2. Since we are interesting in detecting anomalies, the negative class is 'BENIGN'.

In [16]:
X,y=prepare_data(data=df,class_column='Label',classes='binary',neg_class='BENIGN')

### Subset: Dimensionality Reduction w/ PCA

We first implement PCA on the dataset to reduce the number of features. The number of features is informed by the feature ranking provided by the xgboost models trained on the dataset.

In [19]:
new_dimension = 3 #Choose the number of features for PCA

pca_model = PCA(n_components=new_dimension, random_state=4162)
x_red_pca = pca_model.fit_transform(X)
np.shape(x_red_pca)

(529481, 3)

## 1. Training the Isolation Forest Method

We train an Isolation Forest model to detect anomalies in the dataset. Returns -1 for outlier and 1 for inliers.

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html

### Train and save the model

We first train our model and pickle save it for future retrieval. 

The contamination parameter of the model is the expected fraction of outliers in the dataset and should be adjusted if necessary.

In [11]:
isf = IsolationForest(contamination=0,behaviour='new', random_state=42, n_jobs=3)
isf.fit(x_red_pca)

filename='./models/isf_model_'+day_of_week+'_PCA_'+str(new_dimension)+'.pkl'

with open(filename,'wb') as file:
    pickle.dump(isf,file)

## 2. Training the one-class SVM Method

We train an one class SVM model to detect anomalies in the dataset. Returns -1 for outlier and 1 for inliers.

See: https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html#sklearn.svm.OneClassSVM

### Train and save the model

We first train our model and pickle save it for future retrieval. The kernel choice is **Radial Basis Function** ('rbf) as we would like to have locality in the decision boundaries learned by our SVM.

In [6]:
svm_model=OneClassSVM(kernel='rbf',gamma='auto')
svm_model.fit(x_red_pca)

filename='./models/svm_model_'+day_of_week+'_PCA_'+str(new_dimension)+'.pkl'

with open(filename,'wb') as file:
    pickle.dump(svm_model,file)

## 3. Training the Elliptic Envelope method

We train an an Elliptic envelope model designed to detect outliers in a Gaussian distributed dataset.

Returns -1 for outlier and 1 for inliers.

See: https://scikit-learn.org/stable/modules/generated/sklearn.covariance.EllipticEnvelope.html

### Train and save the model

We first train our model and pickle save it for future retrieval. 

The contamination parameter of the model is the expected fraction of outliers in the dataset and should be adjusted if necessary.

In [12]:
cov = EllipticEnvelope(contamination=0.1,random_state=0)
cov.fit(x_red_pca)

filename='./models/cov_model_'+day_of_week+'_PCA_'+str(new_dimension)+'.pkl'

with open(filename,'wb') as file:
    pickle.dump(cov,file)

## 4. Training the Local Outlier Factor (LOF) method

We train an an LOF model to detect anomalies in the dataset.
Returns -1 for outlier and 1 for inliers.

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.LocalOutlierFactor.html

In [20]:
Novelty= True

lof = LocalOutlierFactor(n_neighbors=35, novelty=Novelty,n_jobs=3)

if Novelty:

    lof = lof.fit(x_red_pca)

    filename='./models/lof_model_'+day_of_week+'_PCA_'+str(new_dimension)+'.pkl'

    with open(filename,'wb') as file:
        pickle.dump(lof,file)

else:

    lof_anomalies=lof.fit_predict(x_red_pca)

    filename='./predictions/lof_anomalies_'+day_of_week+'_PCA_'+str(new_dimension)+'.pkl'

    with open(filename,'wb') as file:
        pickle.dump(lof_anomalies,file)

C:\Users\emirk\Anaconda3\envs\insight\lib\site-packages\sklearn\neighbors\lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
